In [ ]:
# Import required libraries
import os

import pandas as pd

from solubilityccs import Fluid

# Set file path
file_path = "/workspaces/SolubilityCCS/Database/solubilityExperimentalData.xlsx"
print(f"Data file: {file_path}")
print(f"File exists: {os.path.exists(file_path)}")

# Check available sheets
if os.path.exists(file_path):
    excel_file = pd.ExcelFile(file_path)
    print(f"Available sheets: {excel_file.sheet_names}")
else:
    print("Excel file not found!")

Data file: /workspaces/SolubilityCCS/Database/solubilityExperimentalData.xlsx
File exists: True
Available sheets: ['Sheet1', 'validation_results (4)', 'Sheet2']


## Data Reading

Reading the Excel file with single-line header format.

In [ ]:
# Read the Excel file with single-line header format
print("Reading Excel file...")
df = pd.read_excel(file_path, header=0)

print(f"DataFrame shape: {df.shape}")
print(f"Column names: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())

# Basic data info
print("\nData Info:")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print("Non-null data summary:")
print(df.isnull().sum())

Reading Excel file...
DataFrame shape: (26, 14)
Column names: ['N', 'Input.Pressure', 'Input.Temperature', 'Input.H2O', 'Input.H2SO4', 'Input.HNO3', 'Result.betta', 'Liq.H2O', 'Liq.H2SO4', 'Liq.HNO3', 'Gas.H2O', 'Gas.H2SO4', 'Gas.HNO3', 'Comment']

First few rows:
   N  Input.Pressure  Input.Temperature  Input.H2O  Input.H2SO4  Input.HNO3  \
0  1           260.0             353.15      20000            0           0   
1  2           140.0             353.15      20000            0           0   
2  3            40.0             353.15      20000            0           0   
3  4           260.0             323.15      20000            0           0   
4  5           140.0             323.15      20000            0           0   

   Result.betta  Liq.H2O  Liq.H2SO4  Liq.HNO3  Gas.H2O  Gas.H2SO4  Gas.HNO3  \
0  double phase      NaN        NaN       NaN   0.0132        NaN       NaN   
1  double phase      NaN        NaN       NaN   0.0095        NaN       NaN   
2  double phase      Na

## Column Identification

Identifying different types of columns based on the single-line header format.

In [3]:
# Identify different types of columns
print("=== COLUMN IDENTIFICATION ===")

# Input composition columns (excluding temperature and pressure)
input_composition_cols = [
    col
    for col in df.columns
    if col.startswith("Input.") and not col.endswith(("Pressure", "Temperature"))
]
print(f"Input composition columns: {input_composition_cols}")

# Condition columns (temperature and pressure)
condition_cols = [
    col
    for col in df.columns
    if col.startswith("Input.") and col.endswith(("Pressure", "Temperature"))
]
print(f"Condition columns: {condition_cols}")

# Result columns
result_cols = [col for col in df.columns if col.startswith("Result.")]
print(f"Result columns: {result_cols}")

# Gas phase columns
gas_cols = [col for col in df.columns if col.startswith("Gas.")]
print(f"Gas phase columns: {gas_cols}")

# Liquid phase columns
liq_cols = [col for col in df.columns if col.startswith("Liq.")]
print(f"Liquid phase columns: {liq_cols}")

print("\n=== SAMPLE DATA INSPECTION ===")
if len(df) > 0:
    sample_row = df.iloc[0]
    print("Sample row (first experiment):")
    for col in condition_cols + input_composition_cols + result_cols:
        if col in df.columns:
            print(f"  {col}: {sample_row[col]}")

    print("\nGas phase sample:")
    for col in gas_cols:
        if col in df.columns:
            print(f"  {col}: {sample_row[col]}")

    print("\nLiquid phase sample:")
    for col in liq_cols:
        if col in df.columns:
            print(f"  {col}: {sample_row[col]}")

print("\n=== DATA QUALITY CHECK ===")
print("Non-null counts for key columns:")
key_cols = condition_cols + input_composition_cols + result_cols + gas_cols + liq_cols
for col in key_cols:
    if col in df.columns:
        non_null_count = df[col].notna().sum()
        print(f"  {col}: {non_null_count}/{len(df)} non-null values")

if "Result.betta" in df.columns:
    print("\nUnique values for Result.betta:")
    print(df["Result.betta"].value_counts())

=== COLUMN IDENTIFICATION ===
Input composition columns: ['Input.H2O', 'Input.H2SO4', 'Input.HNO3']
Condition columns: ['Input.Pressure', 'Input.Temperature']
Result columns: ['Result.betta']
Gas phase columns: ['Gas.H2O', 'Gas.H2SO4', 'Gas.HNO3']
Liquid phase columns: ['Liq.H2O', 'Liq.H2SO4', 'Liq.HNO3']

=== SAMPLE DATA INSPECTION ===
Sample row (first experiment):
  Input.Pressure: 260.0
  Input.Temperature: 353.15
  Input.H2O: 20000
  Input.H2SO4: 0
  Input.HNO3: 0
  Result.betta: double phase

Gas phase sample:
  Gas.H2O: 0.0132
  Gas.H2SO4: nan
  Gas.HNO3: nan

Liquid phase sample:
  Liq.H2O: nan
  Liq.H2SO4: nan
  Liq.HNO3: nan

=== DATA QUALITY CHECK ===
Non-null counts for key columns:
  Input.Pressure: 26/26 non-null values
  Input.Temperature: 26/26 non-null values
  Input.H2O: 26/26 non-null values
  Input.H2SO4: 26/26 non-null values
  Input.HNO3: 26/26 non-null values
  Result.betta: 26/26 non-null values
  Gas.H2O: 9/26 non-null values
  Gas.H2SO4: 5/26 non-null values
 

## Validation Function

Function to validate experimental data against simulation results.

In [ ]:
MAX_RELATIVE_ERROR = 0.1  # Maximum allowed relative error for validation
MAX_ABS_ERROR = 0.5 / 1e6  # Maximum allowed ppm error for validation


def validate_experimental_data(row):
    """
    Validate a single experimental data point against simulation results.

    Args:
        row: A pandas Series containing the experimental data

    Returns:
        dict: Validation results including simulation data and comparisons
    """
    result = {
        "row_index": row.name,
        "error": "None",
        "simulation_successful": False,
        "experimental_betta": None,
        "simulated_betta": None,
        "betta_validation": None,
    }

    try:
        # Extract conditions
        pressure = row["Input.Pressure"]  # bar
        temperature = row["Input.Temperature"]  # K

        # Extract composition (convert ppm to mole fraction)
        composition = {}

        # H2O composition (ppm to mole fraction)
        h2o_ppm = row["Input.H2O"]
        composition["H2O"] = h2o_ppm / 1e6

        # H2SO4 composition
        h2so4_value = row["Input.H2SO4"]
        composition["H2SO4"] = h2so4_value / 1e6

        # HNO3 composition
        hno3_value = row["Input.HNO3"]
        composition["HNO3"] = hno3_value / 1e6

        # Add CO2 to complete the mixture (remainder)
        total_other_components = sum(composition.values())
        if total_other_components < 1.0:
            composition["CO2"] = 1.0 - total_other_components

        if not composition:
            result["error"] = "No valid composition data found"
            return result

        # Create fluid and run simulation
        fluid = Fluid()

        # Add components
        for component, fraction in composition.items():
            if fraction > 1 * 1e-6:
                fluid.add_component(component, fraction)

        # Set conditions
        fluid.set_temperature(temperature)
        fluid.set_pressure(pressure)

        # Run flash calculation
        fluid.calc_vapour_pressure()
        fluid.flash_activity()

        # Get simulation results
        result["simulation_successful"] = True
        result["simulated_betta"] = fluid.betta if hasattr(fluid, "betta") else None
        result["simulated_pressure"] = pressure
        result["simulated_temperature"] = temperature
        result["composition"] = composition

        # Compare with experimental betta
        exp_betta = row["Result.betta"]
        if pd.notna(exp_betta):
            result["experimental_betta"] = exp_betta

            # Parse experimental betta value
            if isinstance(exp_betta, str):
                if exp_betta.lower() == "double phase":
                    # Double phase: betta should be between 0 and 1
                    if (
                        result["simulated_betta"] is not None
                        and 0 < result["simulated_betta"] < 1
                    ):
                        result["betta_validation"] = (
                            f"VALID (simulated "
                            f"{result['simulated_betta']:.4f}, "
                            f"expected double phase)"
                        )
                    else:
                        result["betta_validation"] = (
                            f"INVALID (simulated "
                            f"{result['simulated_betta']:.4f}, "
                            f"expected double phase)"
                        )
                elif exp_betta.lower() == "single phase":
                    # Single phase: betta should be 0 or 1
                    if result["simulated_betta"] is not None and (
                        result["simulated_betta"] == 0.0
                        or result["simulated_betta"] == 1.0
                    ):
                        result["betta_validation"] = (
                            f"VALID (simulated "
                            f"{result['simulated_betta']:.4f}, "
                            f"expected single phase)"
                        )
                    else:
                        result["betta_validation"] = (
                            f"INVALID (simulated "
                            f"{result['simulated_betta']:.4f}, "
                            f"expected single phase)"
                        )
                elif exp_betta.startswith("<"):
                    # Less than comparison
                    threshold = float(exp_betta[1:])
                    if (
                        result["simulated_betta"] is not None
                        and result["simulated_betta"] < threshold
                    ):
                        result["betta_validation"] = (
                            f"VALID (simulated "
                            f"{result['simulated_betta']:.4f}, "
                            f"expected {exp_betta})"
                        )
                    else:
                        result["betta_validation"] = (
                            f"INVALID (simulated "
                            f"{result['simulated_betta']:.4f}, "
                            f"expected {exp_betta})"
                        )
                else:
                    try:
                        exp_betta_num = float(exp_betta)
                        if result["simulated_betta"] is not None:
                            relative_error = abs(
                                result["simulated_betta"] - exp_betta_num
                            ) / max(exp_betta_num, 0.001)
                            if relative_error < 0.05:
                                result["betta_validation"] = (
                                    f"VALID ({relative_error*100:.2f}%, "
                                    f"simulated "
                                    f"{result['simulated_betta']:.4f}, "
                                    f"expected {exp_betta_num:.4f})"
                                )
                            else:
                                result["betta_validation"] = (
                                    f"INVALID ({relative_error*100:.2f}%, "
                                    f"simulated "
                                    f"{result['simulated_betta']:.4f}, "
                                    f"expected {exp_betta_num:.4f})"
                                )
                        else:
                            result["betta_validation"] = (
                                f"INVALID (simulated None, "
                                f"expected {exp_betta_num:.4f})"
                            )
                    except ValueError:
                        result["betta_validation"] = (
                            f"UNKNOWN (simulated "
                            f"{result['simulated_betta']:.4f}, "
                            f"expected {exp_betta})"
                        )
            else:
                # Numeric comparison
                if result["simulated_betta"] is not None:
                    relative_error = abs(result["simulated_betta"] - exp_betta) / max(
                        exp_betta, 0.001
                    )
                    if relative_error < 0.05:
                        result["betta_validation"] = (
                            f"VALID ({relative_error*100:.2f}%, "
                            f"simulated {result['simulated_betta']:.4f}, "
                            f"expected {exp_betta:.4f})"
                        )
                    else:
                        result["betta_validation"] = (
                            f"INVALID ({relative_error*100:.2f}%, "
                            f"simulated {result['simulated_betta']:.4f}, "
                            f"expected {exp_betta:.4f})"
                        )
                else:
                    result["betta_validation"] = (
                        f"INVALID (simulated None, " f"expected {exp_betta:.4f})"
                    )

        # Compare phase-specific components
        if hasattr(fluid, "phases") and len(fluid.phases) >= 2:
            # Try to identify gas and liquid phases
            gas_phase = None
            liquid_phase = None

            gas_phase = fluid.phases[0]
            liquid_phase = fluid.phases[1]

            # Gas phase comparisons
            for col in gas_cols:
                component = col.replace("Gas.", "")
                exp_value = row[col]
                if pd.notna(exp_value) and gas_phase is not None:
                    try:
                        # Find component in gas phase
                        if component in gas_phase.components:
                            comp_idx = gas_phase.components.index(component)
                            sim_value = gas_phase.fractions[comp_idx]
                            relative_error = abs(sim_value - exp_value) / max(
                                exp_value, 1e-15
                            )
                            absolute_error = abs(sim_value - exp_value)
                            if relative_error < 0.1:
                                result[col] = (
                                    f"VALID ({relative_error*100:.2f}%, "
                                    f"simulated {sim_value:.6f}, "
                                    f"expected {exp_value:.6f})"
                                )
                            else:
                                if absolute_error < MAX_ABS_ERROR:
                                    result[col] = (
                                        f"VALID (absolute error "
                                        f"{absolute_error:.6f}, "
                                        f"simulated {sim_value:.6f}, "
                                        f"expected {exp_value:.6f})"
                                    )
                                else:
                                    result[col] = (
                                        f"INVALID (absolute error "
                                        f"{absolute_error:.2f}, "
                                        f"ppm;{relative_error*100:.2f}%, "
                                        f"simulated {sim_value:.6f}, "
                                        f"expected {exp_value:.6f})"
                                    )
                        else:
                            result[col] = (
                                f"COMPONENT_NOT_FOUND " f"(expected {exp_value:.6f})"
                            )
                    except Exception as e:
                        result[col] = f"CALCULATION_ERROR ({str(e)})"

            # Liquid phase comparisons
            for col in liq_cols:
                component = col.replace("Liq.", "")
                exp_value = row[col]
                if pd.notna(exp_value) and liquid_phase is not None:
                    try:
                        # Find component in liquid phase
                        if component in liquid_phase.components:
                            comp_idx = liquid_phase.components.index(component)
                            sim_value = liquid_phase.fractions[comp_idx]
                            relative_error = abs(sim_value - exp_value) / max(
                                exp_value, 1e-15
                            )
                            if relative_error < 0.1:
                                result[col] = (
                                    f"VALID ({relative_error*100:.2f}%, "
                                    f"simulated {sim_value:.6f}, "
                                    f"expected {exp_value:.6f})"
                                )
                            else:
                                absolute_error = abs(sim_value - exp_value)
                                if absolute_error < MAX_ABS_ERROR:
                                    result[col] = (
                                        f"VALID (absolute error "
                                        f"{absolute_error:.6f}, "
                                        f"simulated {sim_value:.6f}, "
                                        f"expected {exp_value:.6f})"
                                    )
                                else:
                                    result[col] = (
                                        f"INVALID (absolute error "
                                        f"{absolute_error:.2f}, "
                                        f"ppm; {relative_error*100:.2f}%, "
                                        f"simulated {sim_value:.6f}, "
                                        f"expected {exp_value:.6f})"
                                    )
                        else:
                            result[col] = (
                                f"COMPONENT_NOT_FOUND " f"(expected {exp_value:.6f})"
                            )
                    except Exception as e:
                        result[col] = f"CALCULATION_ERROR ({str(e)})"

    except Exception as e:
        result["error"] = str(e)
        result["simulation_successful"] = False

    return result


# Test the function with the first row
print("=== TESTING VALIDATION FUNCTION ===")
if len(df) > 0:
    test_row = df.iloc[15]
    test_result = validate_experimental_data(test_row)
    print("Test result:")
    for key, value in test_result.items():
        print(f"  {key}: {value}")

    print("\n=== COMPOSITION ANALYSIS ===")
    print(f"Input H2O: {test_row['Input.H2O']} ppm")
    print(f"Input H2SO4: {test_row['Input.H2SO4']}")
    print(f"Input HNO3: {test_row['Input.HNO3']}")
    print(f"Pressure: {test_row['Input.Pressure']} bar")
    print(f"Temperature: {test_row['Input.Temperature']} K")
else:
    print("No data available for testing")

=== TESTING VALIDATION FUNCTION ===


Test result:
  row_index: 15
  error: None
  simulation_successful: True
  experimental_betta: double phase
  simulated_betta: 0.974731362568491
  betta_validation: VALID (simulated 0.9747, expected double phase)
  simulated_pressure: 98.6
  simulated_temperature: 297.15
  composition: {'H2O': np.float64(0.01), 'H2SO4': np.float64(0.0), 'HNO3': np.float64(0.0078), 'CO2': np.float64(0.9822)}
  Gas.HNO3: INVALID (absolute error 0.00, ppm;34.38%, simulated 0.001411, expected 0.002150)
  Liq.H2O: VALID (0.29%, simulated 0.601761, expected 0.600000)
  Liq.HNO3: VALID (0.06%, simulated 0.398239, expected 0.398000)

=== COMPOSITION ANALYSIS ===
Input H2O: 10000 ppm
Input H2SO4: 0
Input HNO3: 7800
Pressure: 98.6 bar
Temperature: 297.15 K


## Full Validation Workflow

Running validation on all experimental data points.

In [ ]:
# Run validation on all experimental data
print("=== RUNNING FULL VALIDATION ===")
print(f"Processing {len(df)} experimental data points...")

validation_results = []
for idx, row in df.iterrows():
    print(f"Processing row {idx+1}/{len(df)}", end="\r")
    result = validate_experimental_data(row)
    validation_results.append(result)

# Convert results to DataFrame
results_df = pd.DataFrame(validation_results)

print(f"\nValidation complete! Processed {len(results_df)} rows.")
print(f"Successful simulations: {results_df['simulation_successful'].sum()}")
print(f"Failed simulations: {(~results_df['simulation_successful']).sum()}")

# Display summary of errors
if (~results_df["simulation_successful"]).sum() > 0:
    print("\nError summary:")
    error_counts = results_df[results_df["error"] != "None"]["error"].value_counts()
    for error, count in error_counts.items():
        print(f"  {error}: {count} occurrences")

# Display betta validation summary
betta_validation_counts = results_df["betta_validation"].value_counts()
print("\nBetta validation summary:")
for status, count in betta_validation_counts.items():
    print(f"  {status}: {count} cases")

# Display first few results
print("\nFirst 5 validation results:")
display_cols = [
    "row_index",
    "simulation_successful",
    "experimental_betta",
    "simulated_betta",
    "betta_validation",
    "error",
]
print(results_df[display_cols].head())
results_df.to_csv(
    "/workspaces/SolubilityCCS/Database/validation_results.csv", index=False
)

=== RUNNING FULL VALIDATION ===
Processing 26 experimental data points...
Processing row 26/26
Validation complete! Processed 26 rows.
Successful simulations: 26
Failed simulations: 0

Betta validation summary:
  VALID (simulated 1.0000, expected single phase): 3 cases
  VALID (simulated 0.9867, expected double phase): 2 cases
  VALID (simulated 0.9903, expected double phase): 1 cases
  VALID (simulated 0.9688, expected double phase): 1 cases
  VALID (simulated 0.9813, expected double phase): 1 cases
  VALID (simulated 0.9897, expected double phase): 1 cases
  VALID (simulated 0.9953, expected double phase): 1 cases
  VALID (simulated 0.9866, expected double phase): 1 cases
  VALID (simulated 0.9842, expected double phase): 1 cases
  VALID (simulated 0.9854, expected double phase): 1 cases
  VALID (simulated 0.9847, expected double phase): 1 cases
  VALID (simulated 1.0000, expected double phase): 1 cases
  VALID (simulated 0.0001, expected double phase): 1 cases
  VALID (simulated 0.9

## Results Analysis

Detailed analysis of validation results and export to files.

In [ ]:
# Summary and Analysis
print("=== FINAL SUMMARY ===")

# Filter results based on success/failure
successful_results = results_df[results_df["error"] == "None"]
failed_results = results_df[results_df["error"] != "None"]

# Filter results with betta comparison
betta_compared = successful_results[successful_results["experimental_betta"].notna()]
# Extract just the validation status from the detailed string
betta_compared_copy = betta_compared.copy()
betta_compared_copy["betta_status"] = (
    betta_compared_copy["betta_validation"].str.split("(").str[0]
)
valid_betta = betta_compared_copy[betta_compared_copy["betta_status"] == "VALID"]

# Create summary statistics
summary_stats = {
    "total_experiments": len(df),
    "successful_simulations": len(successful_results),
    "failed_simulations": len(failed_results),
    "success_rate": len(successful_results) / len(df) * 100,
    "experiments_with_betta_comparison": len(betta_compared),
    "valid_betta_predictions": len(valid_betta),
    "betta_validation_rate": (
        len(valid_betta) / len(betta_compared) * 100 if len(betta_compared) > 0 else 0
    ),
}

print(f"Total experimental data points: {summary_stats['total_experiments']}")
print(
    f"Successful simulations: {summary_stats['successful_simulations']} "
    f"({summary_stats['success_rate']:.1f}%)"
)
print(
    f"Failed simulations: {summary_stats['failed_simulations']} "
    f"({100 - summary_stats['success_rate']:.1f}%)"
)
print(
    f"Experiments with betta comparison: "
    f"{summary_stats['experiments_with_betta_comparison']}"
)
print(f"Valid betta predictions: {summary_stats['valid_betta_predictions']}")
print(f"Betta validation rate: {summary_stats['betta_validation_rate']:.1f}%")

print("\n=== DETAILED BETTA VALIDATION RESULTS ===")
# Show detailed betta validation results
if len(betta_compared) > 0:
    print("Sample betta validation results:")
    for idx, row in betta_compared.head(10).iterrows():
        print(f"Row {idx}: {row['betta_validation']}")

print("\n=== PHASE COMPONENT COMPARISON SUMMARY ===")
# Count phase-component comparisons
phase_component_results = []
for col in results_df.columns:
    if col.startswith("Gas.") or col.startswith("Liq."):
        component_results = results_df[col].dropna()
        # Extract validation status from detailed strings
        valid_count = sum(1 for x in component_results if x.startswith("VALID"))
        total_count = len(component_results)
        if total_count > 0:
            phase_component_results.append(
                {
                    "component": col,
                    "valid": valid_count,
                    "total": total_count,
                    "rate": valid_count / total_count * 100,
                }
            )

if phase_component_results:
    print(f"{'Component':<15} {'Valid':<8} {'Total':<8} {'Rate (%)':<10}")
    print("-" * 45)
    for result in phase_component_results:
        print(
            f"{result['component']:<15} {result['valid']:<8} "
            f"{result['total']:<8} {result['rate']:<10.1f}"
        )

print("\n=== SAMPLE PHASE COMPONENT RESULTS ===")
# Show some detailed phase component results
for col in results_df.columns:
    if col.startswith("Gas.") or col.startswith("Liq."):
        component_results = results_df[col].dropna()
        if len(component_results) > 0:
            print(f"\n{col} - Sample results:")
            for idx, result in component_results.head(3).items():
                print(f"  Row {idx}: {result}")
            break

print("\n=== COMMON FAILURE REASONS ===")
if len(failed_results) > 0:
    error_counts = failed_results["error"].value_counts()
    for error, count in error_counts.head(10).items():
        print(f"{error}: {count} occurrences")

# Export results
print("\n=== EXPORTING RESULTS ===")
# Export validation results
results_export_path = file_path.replace(".xlsx", "_validation_results.csv")
results_df.to_csv(results_export_path, index=False)
print(f"Validation results exported to: {results_export_path}")

# Export combined dataset
combined_df = pd.concat(
    [df.reset_index(drop=True), results_df.reset_index(drop=True)], axis=1
)
combined_export_path = file_path.replace(".xlsx", "_with_validation.csv")
combined_df.to_csv(combined_export_path, index=False)
print(f"Combined dataset exported to: {combined_export_path}")

print("\n=== VALIDATION COMPLETE ===")
print("Check the notebook output above for detailed validation results.")
print("The exported CSV files contain:")
print(
    "- Validation results: simulation outcomes and comparisons "
    "with detailed error information"
)
print("- Combined dataset: original data with validation results")
print("\nKey findings:")
print("- Review the success rate of simulations")
print("- Check cases where betta validation failed " "with specific error percentages")
print("- Examine phase-component comparison results " "with detailed error analysis")
print("- Use the exported files for further analysis")
print("\nNote: All validation results now include detailed error information:")
print("- Relative error percentages for numerical comparisons")
print("- Simulated vs expected values for easy comparison")
print("- Specific error messages for failed calculations")

=== FINAL SUMMARY ===
Total experimental data points: 26
Successful simulations: 26 (100.0%)
Failed simulations: 0 (0.0%)
Experiments with betta comparison: 26
Valid betta predictions: 0
Betta validation rate: 0.0%

=== DETAILED BETTA VALIDATION RESULTS ===
Sample betta validation results:
Row 0: VALID (simulated 0.9903, expected double phase)
Row 1: VALID (simulated 0.9897, expected double phase)
Row 2: VALID (simulated 0.9953, expected double phase)
Row 3: VALID (simulated 0.9688, expected double phase)
Row 4: VALID (simulated 0.9813, expected double phase)
Row 5: VALID (simulated 0.9842, expected double phase)
Row 6: VALID (simulated 0.9866, expected double phase)
Row 7: VALID (simulated 0.9847, expected double phase)
Row 8: VALID (simulated 0.9854, expected double phase)
Row 9: VALID (simulated 1.0000, expected single phase)

=== PHASE COMPONENT COMPARISON SUMMARY ===
Component       Valid    Total    Rate (%)  
---------------------------------------------
Gas.H2O         9       